### ETL For the Project:
- Source Data avilable in HDFS directory: /home/hadoop/ETL_Project_SRC_data/
- Setting up Spark session from spark context as 'spark'

In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1651684218980_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
hdfs_path = "/home/hadoop/ETL_Project_SRC_data/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Reading data from HDFS into Spark Dataframe using DataFrame Reader API form PySpark

In [3]:
atm_trans = spark.read.format("csv").option("header","true").load(hdfs_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
atm_trans.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- 2017: string (nullable = true)
 |-- January: string (nullable = true)
 |-- 12: string (nullable = true)
 |-- Sunday: string (nullable = true)
 |-- 0: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- 16: string (nullable = true)
 |-- NCR: string (nullable = true)
 |-- NÃƒÂ¦stved: string (nullable = true)
 |-- Farimagsvej: string (nullable = true)
 |-- 8: string (nullable = true)
 |-- 4700: string (nullable = true)
 |-- 55.233: string (nullable = true)
 |-- 11.763: string (nullable = true)
 |-- DKK: string (nullable = true)
 |-- MasterCard: string (nullable = true)
 |-- 5643: string (nullable = true)
 |-- Withdrawal: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- 55.230: string (nullable = true)
 |-- 11.761: string (nullable = true)
 |-- 2616038: string (nullable = true)
 |-- Naestved: string (nullable = true)
 |-- 281.150: string (nullable = true)
 |-- 1014: string (nullable = true)
 |-- 87: string (null

In [5]:
print((atm_trans.count(), len(atm_trans.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(2468571, 34)

This dataset comprises around 2.5 million records and 34 columns.

In [6]:
#Schema defination

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
atm_trans_schema = StructType(fields = [
    StructField("year",IntegerType()) ,
    StructField("month",StringType()),
    StructField("day",IntegerType()),
    StructField("weekday",StringType()),
    StructField("hour",IntegerType()),
    StructField("atm_status",StringType()),
    StructField("atm_id",IntegerType()),
    StructField("atm_manufacturer",StringType()),
    StructField("atm_location",StringType()),
    StructField("atm_streetname",StringType()),
    StructField("atm_street_number",IntegerType()),
    StructField("atm_zipcode",IntegerType()),
    StructField("atm_lat",DoubleType()),
    StructField("atm_lon",DoubleType()),
    StructField("currency",StringType()),
    StructField("card_type",StringType()),
    StructField("message_code",StringType()),
    StructField("service",StringType()),
    StructField("message_text",StringType()),
    StructField("message_text_1",StringType()),
    StructField("weather_lat",DoubleType()),
    StructField("weather_lon",DoubleType()),
    StructField("weather_city_id",IntegerType()),
    StructField("weather_city_name",StringType()),
    StructField("temp",DoubleType()),
    StructField("pressure",IntegerType()),
    StructField("humidity",IntegerType()),
    StructField("wind_speed",IntegerType()),
    StructField("wind_deg",IntegerType()),
    StructField("rain_3h",DoubleType()),
    StructField("clouds_all",IntegerType()),
    StructField("weather_id",IntegerType()),
    StructField("weather_main",StringType()),
    StructField("weather_description",StringType())
    
])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Reading the data again with the defined Schema.
atm_trans = spark.read.format("csv") \
.schema(atm_trans_schema) \
.load(hdfs_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
print(atm_trans.show(5))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+----------+------------+----------+------------+--------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+--------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency| card_type|message_code|   service|message_text|message_text_1|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+----------+------------+----------+------------+--------------+-----------+-



## Creating Dimension tables 
### Location Dimension: 
    - location_id : weather_city_id 
    - location : location name: weather city name
    - street_name : atm_streetname
    - street_number : atm_street_number
    - atm_lat
    - atm_lon

In [9]:
location_dim = atm_trans.select(atm_trans.weather_city_id.alias("location_id"),
                               atm_trans.weather_city_name.alias("location"),
                               atm_trans.atm_streetname.alias("street_name"),
                               atm_trans.atm_street_number.alias("street_number"),
                               atm_trans.atm_zipcode.alias("zipcode"),
                               atm_trans.atm_lat.alias("latitude"),
                               atm_trans.atm_lon.alias("longitude"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
location_dim.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------+-------------------+-------------+-------+--------+---------+
|location_id|      location|        street_name|street_number|zipcode|latitude|longitude|
+-----------+--------------+-------------------+-------------+-------+--------+---------+
|    2616038|      Naestved|        Farimagsvej|            8|   4700|  55.233|   11.763|
|    2616235|NÃƒÂ¸rresundby|         Hadsundvej|           20|   9000|  57.043|     9.95|
|    2616235|NÃƒÂ¸rresundby|         Hadsundvej|           20|   9000|  57.043|     9.95|
|    2619426|         Ikast|RÃƒÂ¥dhusstrÃƒÂ¦det|           12|   7430|  56.139|    9.154|
|    2614481|      Roskilde|       BrÃƒÂ¸nsager|            1|   4000|  55.634|   12.018|
+-----------+--------------+-------------------+-------------+-------+--------+---------+
only showing top 5 rows

In [11]:
location_dim.count() 
# there are around 2.5 million records, however since we derieved this information from a transaction table, there are going to be duplicates.


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [12]:
# Checking for duplicate entries in location dimension table.

## For using SQL, we need to create a temporary view,
location_dim.createOrReplaceTempView("location_dim_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
spark.sql("SELECT location_id, count(*) as count FROM location_dim_table GROUP BY location_id").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+
|location_id| count|
+-----------+------+
|    2624886|280276|
|    2622447| 30741|
|       null|  8087|
|    2612500| 51720|
|    2614010| 18715|
|    2621942|  9119|
|    2612301| 59280|
|    2610095| 36660|
|    2620141| 36847|
|    2620425| 17680|
|    2620952|105574|
|    2616483| 18570|
|    2620279| 36644|
|    2614481| 69989|
|    2610196| 16451|
|    2619771| 26331|
|    2621951| 22318|
|    2617467| 48146|
|    2624602| 36097|
|    2625037| 38775|
+-----------+------+
only showing top 20 rows

In [14]:
spark.sql('SELECT COUNT(DISTINCT(location_id)) FROM location_dim_table').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+
|count(DISTINCT location_id)|
+---------------------------+
|                         52|
+---------------------------+

Clearly, we see that there are 52 dictinct location ids(51 to be precise as NULL is also accounted) in our transaction table.
We also have a considerable NULL records which we donot want in the location dimension table.

In [15]:
location_dim_distinct = location_dim.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
location_dim_distinct.createOrReplaceTempView("location_dim_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
spark.sql("SELECT location_id,location,zipcode, count(*) as count FROM location_dim_table WHERE location_id IS NOT NULL GROUP BY location_id, location,zipcode ORDER BY COUNT(*) DESC").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------------+-------+-----+
|location_id|         location|zipcode|count|
+-----------+-----------------+-------+-----+
|    2624886|          Aalborg|   9000|    6|
|    2620214|         Hjorring|   9800|    3|
|    2612021|        Svenstrup|   9200|    2|
|    2613460|         Slagelse|   4200|    2|
|    2610613|            Vejle|   7100|    2|
|    2614481|         Roskilde|   4000|    2|
|    2615964|    Nykobing Mors|   7870|    2|
|    2621927|    Frederikshavn|   9900|    2|
|    2616235|   NÃƒÂ¸rresundby|   9000|    2|
|    2620952|          Hadsund|   9560|    2|
|    2621394|Gladsakse Kommune|   3400|    2|
|    2624652|            Arhus|   8362|    2|
|    2621927|    Frederikshavn|   9300|    2|
|    2618425|       Copenhagen|   1550|    2|
|    2624652|            Arhus|   8000|    2|
|    2620046|        Holstebro|   7500|    1|
|    2616483|             Nibe|   9240|    1|
|    2620473|        Helsingor|   3000|    1|
|    2617467|          Logstor|   

In [18]:
spark.sql("SELECT * FROM location_dim_table WHERE location_id = '2624886'").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+------------+-------------+-------+--------+---------+
|location_id|location| street_name|street_number|zipcode|latitude|longitude|
+-----------+--------+------------+-------------+-------+--------+---------+
|    2624886| Aalborg| Vestre Alle|            2|   9000|  57.034|    9.908|
|    2624886| Aalborg|ÃƒËœsterÃƒÂ¥|           12|   9000|  57.049|    9.922|
|    2624886| Aalborg|Europa Plads|            4|   9000|  57.044|    9.913|
|    2624886| Aalborg| Hasserisvej|          113|   9000|  57.044|    9.898|
|    2624886| Aalborg| Skelagervej|           15|   9000|  57.023|    9.891|
|    2624886| Aalborg|      Torvet|            6|   9400|  57.059|    9.922|
|    2624886| Aalborg|   Kastetvej|           36|   9000|  57.053|    9.905|
+-----------+--------+------------+-------------+-------+--------+---------+

In [19]:
location_dim_distinct.filter(location_dim_distinct.location_id =='null').show()
#dataframe.filter(dataframe.college != "IIT"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+-----------+-------------+-------+--------+---------+
|location_id|location|street_name|street_number|zipcode|latitude|longitude|
+-----------+--------+-----------+-------------+-------+--------+---------+
+-----------+--------+-----------+-------------+-------+--------+---------+

In [20]:
# There are many null values in the rows of our dataset, checking the percentage of NULLs

location_dim_distinct.show(truncate = False)
# Since Location ID is our key in the dimensional model we have to remove the nulls

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------+-----------------+-------------+-------+--------+---------+
|location_id|location      |street_name      |street_number|zipcode|latitude|longitude|
+-----------+--------------+-----------------+-------------+-------+--------+---------+
|2620214    |Hjorring      |ÃƒËœstergade     |8            |9800   |57.459  |9.988    |
|2622310    |Farso         |Torvet           |8            |9640   |56.771  |9.34     |
|null       |null          |Slotsvolden      |7            |4300   |55.718  |11.704   |
|null       |null          |NÃƒÂ¸rrebro      |18           |9881   |57.541  |10.2     |
|null       |null          |KÃƒÂ¸benhavnsvej |31           |3400   |55.933  |12.314   |
|2618425    |Copenhagen    |Regnbuepladsen   |5            |1550   |55.676  |12.571   |
|null       |null          |Hostrupsvej      |6            |7500   |56.373  |8.625    |
|null       |null          |GodthÃƒÂ¥bsvej   |14           |9230   |56.973  |9.851    |
|null       |null          |Sct.

In [21]:
# Removing the Rows containing nulls
location_dim_distinct_not_null = location_dim_distinct.na.drop(how = 'any' )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
location_dim_distinct_not_null.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------------+----------------+-------------+-------+--------+---------+
|location_id|location         |street_name     |street_number|zipcode|latitude|longitude|
+-----------+-----------------+----------------+-------------+-------+--------+---------+
|2620214    |Hjorring         |ÃƒËœstergade    |8            |9800   |57.459  |9.988    |
|2622310    |Farso            |Torvet          |8            |9640   |56.771  |9.34     |
|2618425    |Copenhagen       |Regnbuepladsen  |5            |1550   |55.676  |12.571   |
|2613460    |Slagelse         |Mariendals Alle |29           |4200   |55.398  |11.342   |
|2618528    |Kolding          |Vejlevej        |135          |6000   |55.505  |9.457    |
|2620952    |Hadsund          |Bymidten        |2            |9575   |56.815  |10.057   |
|2624886    |Aalborg          |Vestre Alle     |2            |9000   |57.034  |9.908    |
|2618468    |Kongens Lyngby   |Jernbanevej     |6            |2800   |55.772  |12.5     |
|2612045  

In [23]:
location_dim_distinct_not_null.createOrReplaceTempView("location_dim_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
spark.sql("SELECT location_id,location,zipcode, count(*) as count FROM location_dim_table WHERE location_id IS NOT NULL GROUP BY location_id, location,zipcode ORDER BY COUNT(*) DESC").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------------+-------+-----+
|location_id|         location|zipcode|count|
+-----------+-----------------+-------+-----+
|    2624886|          Aalborg|   9000|    6|
|    2620214|         Hjorring|   9800|    3|
|    2624652|            Arhus|   8000|    2|
|    2610613|            Vejle|   7100|    2|
|    2616235|   NÃƒÂ¸rresundby|   9000|    2|
|    2621927|    Frederikshavn|   9300|    2|
|    2615964|    Nykobing Mors|   7870|    2|
|    2621927|    Frederikshavn|   9900|    2|
|    2624652|            Arhus|   8362|    2|
|    2621394|Gladsakse Kommune|   3400|    2|
|    2618425|       Copenhagen|   1550|    2|
|    2614481|         Roskilde|   4000|    2|
|    2612021|        Svenstrup|   9200|    2|
|    2613460|         Slagelse|   4200|    2|
|    2620952|          Hadsund|   9560|    2|
|    2624652|            Arhus|   8270|    1|
|    2616483|             Nibe|   9240|    1|
|    2614010|           Sindal|   9881|    1|
|    2610823|            Vadum|   

**There are still location ids which have multiple values like:**

    2624886,
    2620214,
    2615964,
    2613460,
    2620952,
    2621394,
    2616235,
    2621927,
    2610613,
    2624652,
    2618425,
    2621927,
    2614481,
    2612021,
    2624652



In [25]:
spark.sql("SELECT * FROM location_dim_table WHERE location_id in (2624886)").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+------------+-------------+-------+--------+---------+
|location_id|location| street_name|street_number|zipcode|latitude|longitude|
+-----------+--------+------------+-------------+-------+--------+---------+
|    2624886| Aalborg| Vestre Alle|            2|   9000|  57.034|    9.908|
|    2624886| Aalborg|ÃƒËœsterÃƒÂ¥|           12|   9000|  57.049|    9.922|
|    2624886| Aalborg|Europa Plads|            4|   9000|  57.044|    9.913|
|    2624886| Aalborg| Hasserisvej|          113|   9000|  57.044|    9.898|
|    2624886| Aalborg| Skelagervej|           15|   9000|  57.023|    9.891|
|    2624886| Aalborg|      Torvet|            6|   9400|  57.059|    9.922|
|    2624886| Aalborg|   Kastetvej|           36|   9000|  57.053|    9.905|
+-----------+--------+------------+-------------+-------+--------+---------+

**Observations :**
* We can see from the data that there are multiple street_names in a particular location and as per schema we cannot have location_id as a primary key, we can have it as non nullable key.

* For Primary Key, more ideal would be to have a composite key made of : location_id,loaction,street_number.

In [26]:
# Storing the final location_dim_distinct_not_null dataframe as Location_dim
location_dim = location_dim_distinct_not_null

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
location_dim.explain()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan(isFinalPlan=false)
+- HashAggregate(keys=[zipcode#326, location#323, latitude#327, longitude#328, street_name#324, location_id#322, street_number#325], functions=[])
   +- Exchange hashpartitioning(zipcode#326, location#323, latitude#327, longitude#328, street_name#324, location_id#322, street_number#325, 1000)
      +- HashAggregate(keys=[zipcode#326, location#323, latitude#327, longitude#328, street_name#324, location_id#322, street_number#325], functions=[])
         +- Project [weather_city_id#139 AS location_id#322, weather_city_name#140 AS location#323, atm_streetname#126 AS street_name#324, atm_street_number#127 AS street_number#325, atm_zipcode#128 AS zipcode#326, atm_lat#129 AS latitude#327, atm_lon#130 AS longitude#328]
            +- Filter AtLeastNNulls(n, weather_city_id#139,weather_city_name#140,atm_streetname#126,atm_street_number#127,atm_zipcode#128,atm_lat#129,atm_lon#130)
               +- FileScan csv [atm_streetname#126,atm_stre

### * Date Dimension:
    - date_id : year +month+day
    - full_date_time
    - year : year
    - month : month
    - day : day
    - Weekday: weekday
    
Date_id needs to be a derieved column in Source_table and same in Dimension table so we can perform join on that column.
year+month+day could be a good candidate for it:
 - Create a date_id in source table by combining year + month + day
 - converting the resulting date_id column to DATETIME format.

In [28]:
from pyspark.sql.functions import concat,lit

atm_trans = atm_trans.withColumn("date_id", concat(atm_trans.year,lit('/'),atm_trans.month[0:3],lit('/'),atm_trans.day))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
atm_trans.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: integer (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- service: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- message_text_1: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: string

In [30]:
atm_trans.select("date_id").dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('date_id', 'string')]

In [31]:
atm_trans.select('date_id').show(4,truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|date_id   |
+----------+
|2017/Jan/1|
|2017/Jan/1|
|2017/Jan/1|
|2017/Jan/1|
+----------+
only showing top 4 rows

In [32]:
# Convert String to Date Object:
from pyspark.sql.functions import to_date,to_timestamp,col

atm_trans = atm_trans.withColumn('date_id',to_date(col("date_id"),"yyyy/MMM/dd"))

atm_trans.printSchema()
atm_trans.select("date_id").dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: integer (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- service: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- message_text_1: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: string

In [33]:
atm_trans.select('date_id').show(4,truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|date_id   |
+----------+
|2017-01-01|
|2017-01-01|
|2017-01-01|
|2017-01-01|
+----------+
only showing top 4 rows

In [34]:
# Creating Date Dimension:

date_dim = atm_trans.select(atm_trans.date_id .alias("date_id"),
                               atm_trans.year.alias("year"),
                               atm_trans.month.alias("month"),
                               atm_trans.day.alias("day"),
                               atm_trans.weekday.alias("weekday"),
                               atm_trans.hour.alias("hour"))
                               


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
date_dim = date_dim.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
date_dim.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+--------+---+---------+----+
|   date_id|year|   month|day|  weekday|hour|
+----------+----+--------+---+---------+----+
|2017-01-11|2017| January| 11|Wednesday|  23|
|2017-01-06|2017| January|  6|   Friday|  11|
|2017-02-17|2017|February| 17|   Friday|   2|
|2017-01-29|2017| January| 29|   Sunday|   5|
|2017-01-05|2017| January|  5| Thursday|  11|
|2017-02-02|2017|February|  2| Thursday|   1|
|2017-02-20|2017|February| 20|   Monday|   2|
|2017-01-09|2017| January|  9|   Monday|   5|
|2017-01-10|2017| January| 10|  Tuesday|  13|
|2017-02-19|2017|February| 19|   Sunday|  10|
|2017-02-01|2017|February|  1|Wednesday|   5|
|2017-01-25|2017| January| 25|Wednesday|   4|
|2017-02-03|2017|February|  3|   Friday|  15|
|2017-01-28|2017| January| 28| Saturday|   6|
|2017-01-30|2017| January| 30|   Monday|  19|
|2017-01-20|2017| January| 20|   Friday|  15|
|2017-01-04|2017| January|  4|Wednesday|  13|
|2017-01-06|2017| January|  6|   Friday|  17|
|2017-02-12|2017|February| 12|   S

In [37]:
## For using SQL, we need to create a temporary view,
date_dim.createOrReplaceTempView("date_dim_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
spark.sql("SELECT date_id,count(*) FROM date_dim_table GROUP BY date_id ORDER BY COUNT(*) DESC ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+
|   date_id|count(1)|
+----------+--------+
|2017-11-04|      24|
|2017-08-09|      24|
|2017-08-15|      24|
|2017-03-31|      24|
|2017-07-12|      24|
|2017-09-12|      24|
|2017-11-20|      24|
|2017-10-06|      24|
|2017-08-06|      24|
|2017-03-06|      24|
|2017-11-08|      24|
|2017-09-22|      24|
|2017-03-29|      24|
|2017-07-27|      24|
|2017-10-12|      24|
|2017-08-25|      24|
|2017-11-14|      24|
|2017-07-05|      24|
|2017-09-17|      24|
|2017-05-17|      24|
+----------+--------+
only showing top 20 rows

In [39]:
date_dim = date_dim.withColumn("date_time_stamp", concat(date_dim.date_id,lit(' '),date_dim.hour,lit(':00'),lit(':00')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
date_dim.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+--------+---+---------+----+-------------------+
|   date_id|year|   month|day|  weekday|hour|    date_time_stamp|
+----------+----+--------+---+---------+----+-------------------+
|2017-01-11|2017| January| 11|Wednesday|  23|2017-01-11 23:00:00|
|2017-01-06|2017| January|  6|   Friday|  11|2017-01-06 11:00:00|
|2017-02-17|2017|February| 17|   Friday|   2| 2017-02-17 2:00:00|
|2017-01-29|2017| January| 29|   Sunday|   5| 2017-01-29 5:00:00|
|2017-01-05|2017| January|  5| Thursday|  11|2017-01-05 11:00:00|
+----------+----+--------+---+---------+----+-------------------+
only showing top 5 rows

In [41]:
date_dim = date_dim.withColumn('date_time_stamp', to_timestamp(col('date_time_stamp'), 'yyyy-MM-dd HH:mm:ss'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
date_dim.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date_id: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- date_time_stamp: timestamp (nullable = true)

In [43]:
date_dim.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+------+---+---------+----+-------------------+
|   date_id|year| month|day|  weekday|hour|    date_time_stamp|
+----------+----+------+---+---------+----+-------------------+
|2017-07-04|2017|  July|  4|  Tuesday|  16|2017-07-04 16:00:00|
|2017-07-23|2017|  July| 23|   Sunday|  12|2017-07-23 12:00:00|
|2017-08-02|2017|August|  2|Wednesday|  20|2017-08-02 20:00:00|
|2017-07-13|2017|  July| 13| Thursday|   4|2017-07-13 04:00:00|
|2017-07-22|2017|  July| 22| Saturday|  13|2017-07-22 13:00:00|
+----------+----+------+---+---------+----+-------------------+
only showing top 5 rows

### * ATM dimension:
    - ATM ID
    - ATM Number
    - ATM manufacturer
    - ATM Location_id
    - ATM Location
    





In [44]:



atm_dim = atm_trans.select(atm_trans.atm_id .alias("atm_id"),
                               concat(atm_trans.atm_street_number,lit('_'),atm_trans.atm_streetname).alias("atm_number"),
                               atm_trans.atm_manufacturer.alias("atm_manufacturer"),
                               atm_trans.weather_city_id.alias("atm_loaction_id"),
                               atm_trans.atm_location.alias("atm_location")
                               #atm_trans.atm_streetname.alias("atm_streetname"),
                               #atm_trans.atm_status.alias("atm_status"),
                               #atm_trans.atm_zipcode.alias("atm_zipcode"),
                               #atm_trans.atm_lat.alias("atm_lat"),
                               #atm_trans.atm_lon.alias("atm_lon")
                          )
                               

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
atm_dim = atm_dim.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
atm_dim.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

225

In [47]:
atm_dim.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------------------+----------------+---------------+--------------------+
|atm_id|atm_number           |atm_manufacturer|atm_loaction_id|atm_location        |
+------+---------------------+----------------+---------------+--------------------+
|9     |12_Storegade         |Diebold Nixdorf |2620952        |Hadsund             |
|95    |75_RÃƒÂ¥dhuspladsen  |NCR             |2618425        |Intern  KÃƒÂ¸benhavn|
|19    |135_Vejlevej         |NCR             |2618528        |Kolding             |
|17    |16_ÃƒËœstervold      |NCR             |2620952        |Randers             |
|39    |14_GodthÃƒÂ¥bsvej    |NCR             |null           |Svenstrup           |
|10    |6_Torvet             |NCR             |null           |NÃƒÂ¸rresundby      |
|58    |4_Baunebakken        |NCR             |2612500        |StÃƒÂ¸vring         |
|96    |4_Europa Plads       |NCR             |null           |Aalborg Hallen      |
|111   |75_Ceres Byen        |Diebold Nixdorf |2624652        |Aa

In [48]:
# We could see unwanted Null Values, due to atm_location_id ; treating the null values:
atm_dim = atm_dim.na.drop(how = 'any' )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
## For using SQL, we need to create a temporary view,
atm_dim.createOrReplaceTempView("atm_dim_table")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
spark.sql("SELECT atm_id,count(*) FROM atm_dim_table GROUP BY atm_id ORDER BY COUNT(*) DESC ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+
|atm_id|count(1)|
+------+--------+
|    81|       1|
|    65|       1|
|    76|       1|
|    12|       1|
|   110|       1|
|    11|       1|
|    83|       1|
|    33|       1|
|    60|       1|
|    87|       1|
|    90|       1|
|    66|       1|
|   104|       1|
|    99|       1|
|    74|       1|
|     4|       1|
|    35|       1|
|   107|       1|
|     8|       1|
|     1|       1|
+------+--------+
only showing top 20 rows

In [51]:
atm_dim.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------------------+----------------+---------------+--------------------------+
|atm_id|atm_number            |atm_manufacturer|atm_loaction_id|atm_location              |
+------+----------------------+----------------+---------------+--------------------------+
|9     |12_Storegade          |Diebold Nixdorf |2620952        |Hadsund                   |
|95    |75_RÃƒÂ¥dhuspladsen   |NCR             |2618425        |Intern  KÃƒÂ¸benhavn      |
|19    |135_Vejlevej          |NCR             |2618528        |Kolding                   |
|17    |16_ÃƒËœstervold       |NCR             |2620952        |Randers                   |
|111   |75_Ceres Byen         |Diebold Nixdorf |2624652        |Aarhus                    |
|58    |4_Baunebakken         |NCR             |2612500        |StÃƒÂ¸vring               |
|108   |53_Toftevej           |NCR             |2624652        |HÃƒÂ¸rning Hallen         |
|92    |8_ÃƒËœstergade        |NCR             |2620214        |Intern  HjÃƒÂ¸rr

### * Card type dimension

We can have a card_number and card_type associated with it.

A card_type_id to be associated with each transaction.
        - 

In [52]:
unique_cards = atm_trans.select("card_type").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
unique_cards.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           card_type|
+--------------------+
|Visa Dankort - on-us|
|  Mastercard - on-us|
|         HÃƒÂ¦vekort|
|            VisaPlus|
|     Dankort - on-us|
|        Visa Dankort|
| HÃƒÂ¦vekort - on-us|
|              CIRRUS|
|                VISA|
|             Maestro|
|          MasterCard|
|             Dankort|
+--------------------+

In [54]:

from pyspark.sql.functions import when
atm_trans = atm_trans.withColumn('card_type_id', 
                    when(atm_trans.card_type == "Visa Dankort - on-us", 1)
                   .when(atm_trans.card_type == "Mastercard - on-us", 2)
                   .when(atm_trans.card_type == "HÃƒÂ¦vekort", 3)
                   .when(atm_trans.card_type == "VisaPlus", 4)
                   .when(atm_trans.card_type == "Dankort - on-us", 5)
                   .when(atm_trans.card_type == "Visa Dankort", 6)
                    .when(atm_trans.card_type == "HÃƒÂ¦vekort - on-us", 7)
                    .when(atm_trans.card_type == "CIRRUS", 8)
                    .when(atm_trans.card_type == "VISA", 9)
                    .when(atm_trans.card_type == "Maestro", 10)
                    .when(atm_trans.card_type == "MasterCard", 11)
                    .when(atm_trans.card_type == "Dankort", 12)
           .otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
#cards_dictionary = card_dict(unique_cards)

card_type_dim = atm_trans.select(atm_trans.card_type_id.alias("card_type_id"),
    atm_trans.card_type.alias("card_type"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
card_type_dim.printSchema()

card_type_dim.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- card_type_id: integer (nullable = false)
 |-- card_type: string (nullable = true)

+------------+--------------------+
|card_type_id|           card_type|
+------------+--------------------+
|          11|          MasterCard|
|          11|          MasterCard|
|           9|                VISA|
|           9|                VISA|
|          11|          MasterCard|
|          11|          MasterCard|
|          11|          MasterCard|
|           2|  Mastercard - on-us|
|          11|          MasterCard|
|           9|                VISA|
|          12|             Dankort|
|           6|        Visa Dankort|
|           2|  Mastercard - on-us|
|           2|  Mastercard - on-us|
|           2|  Mastercard - on-us|
|           1|Visa Dankort - on-us|
|          11|          MasterCard|
|           2|  Mastercard - on-us|
|           6|        Visa Dankort|
|           9|                VISA|
+------------+--------------------+
only showing top 20 rows

In [57]:
#Removing the duplicate rows:
card_type_dim = card_type_dim.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
card_type_dim.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|card_type_id|card_type           |
+------------+--------------------+
|10          |Maestro             |
|8           |CIRRUS              |
|4           |VisaPlus            |
|2           |Mastercard - on-us  |
|3           |HÃƒÂ¦vekort         |
|7           |HÃƒÂ¦vekort - on-us |
|5           |Dankort - on-us     |
|11          |MasterCard          |
|6           |Visa Dankort        |
|9           |VISA                |
|1           |Visa Dankort - on-us|
|12          |Dankort             |
+------------+--------------------+

### Fact Table Schema:

- trans_id # Not available in source data; need to generate an autoincrement series to associate with each transaction
- atm_id
- weather_loc_id : weather_city_id
- date_id
- card_type_id
- atm_status
- currency
- service
- transaction_amount : # Not avaialble in source data ; have to use random numbers as per instructions of assignment
- message_code
- message_text
- rain_3h
- clouds_all
- weather_id
- weather_main
- weather_description



In [59]:
atm_trans.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: integer (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- service: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- message_text_1: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: string

In [60]:
from pyspark.sql.functions import monotonically_increasing_id

atm_trans_fact = atm_trans.select(monotonically_increasing_id().alias('trans_id'),
                               atm_trans.atm_id.alias("atm_id"),
                               atm_trans.weather_city_id.alias("weather_loc_id"),
                               atm_trans.date_id.alias("date_id"),
                               atm_trans.card_type_id.alias("card_type_id"),
                               atm_trans.atm_status.alias("atm_status"),
                               atm_trans.currency.alias("currency"),
                               atm_trans.service.alias("service"),
                               atm_trans.message_code.alias("message_code"),
                               concat(atm_trans.message_text,lit(' | '),atm_trans.message_text_1).alias("message_text"),
                               atm_trans.rain_3h.alias("rain_3h"),
                               atm_trans.clouds_all.alias("clouds_all"),
                               atm_trans.weather_id.alias("weather_id"),
                               atm_trans.weather_main.alias("weather_main"),
                               atm_trans.weather_description.alias("weather_description"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
atm_trans_fact.show(5,truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------------+----------+------------+----------+--------+----------+------------+------------+-------+----------+----------+------------+-----------------------+
|trans_id|atm_id|weather_loc_id|date_id   |card_type_id|atm_status|currency|service   |message_code|message_text|rain_3h|clouds_all|weather_id|weather_main|weather_description    |
+--------+------+--------------+----------+------------+----------+--------+----------+------------+------------+-------+----------+----------+------------+-----------------------+
|0       |1     |2616038       |2017-01-01|11          |Active    |DKK     |Withdrawal|5643        |null        |0.215  |92        |500       |Rain        |light rain             |
|1       |2     |2616235       |2017-01-01|11          |Inactive  |DKK     |Withdrawal|1764        |null        |0.59   |92        |500       |Rain        |light rain             |
|2       |2     |2616235       |2017-01-01|9           |Inactive  |DKK     |Withdrawal|1891    

In [62]:
# Adding transaction_amount: randomly generated
import random
from pyspark.sql.functions import rand
atm_trans_fact = atm_trans_fact.withColumn('trans_amnt', rand()*10000)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
atm_trans_fact.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trans_id: long (nullable = false)
 |-- atm_id: integer (nullable = true)
 |-- weather_loc_id: integer (nullable = true)
 |-- date_id: date (nullable = true)
 |-- card_type_id: integer (nullable = false)
 |-- atm_status: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- rain_3h: double (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)
 |-- trans_amnt: double (nullable = false)

In [64]:

atm_trans_fact.select('trans_amnt').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|        trans_amnt|
+------------------+
| 8713.307781156414|
| 9896.652244462037|
| 7474.364063089788|
| 788.7471046184858|
| 7326.886744588837|
|  3104.92840755795|
|7523.0208057110685|
| 4281.294715647411|
| 6370.106498940855|
| 8619.696539750874|
| 525.0170443352042|
| 5392.196060977626|
| 9922.332980605446|
|1246.7485765261522|
| 8651.691221374474|
|179.31137004770113|
|3907.8256604914463|
| 8226.834906324299|
| 8046.308926717388|
| 5815.106687966652|
+------------------+
only showing top 20 rows

### Now we have our dimension and Fact tables created, we need to store then to S3 object Storage

##### Dimension tables:
- atm_dim
- card_type_dim
- date_dim
- location_dim

##### Fact table:
- atm_trans_fact

In [65]:
s3_bucket = 'etl-proj'
s3_path = 's3://s3_bucket/'


atm_dim.write.format('csv').option('header','true').save('s3a://etl-proj/atm-dimension/atm_dim.csv',mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
card_type_dim.write.format('csv').option('header','true').save('s3a://etl-proj/card-type-dimension/card_type_dim.csv',mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
date_dim.write.format('csv').option('header','true').save('s3a://etl-proj/date-dimension/date_dim.csv',mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
location_dim.write.format('csv').option('header','true').save('s3a://etl-proj/location-dimension/location_dim.csv',mode='overwrite')

##### Writing Fact Table:

In [ ]:
atm_trans_fact.write.format('csv').option('header','true').save('s3a://etl-proj/transaction-fact/atm_trans_fact.csv',mode='overwrite')